<a href="https://colab.research.google.com/github/yorkjong/news-digest/blob/main/notebooks/news_post_to_discord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install

In [ ]:
# @title Download Packages
import os

fns = ['clip.py', 'op.py', 'hashtag.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
    url = f'https://raw.githubusercontent.com/yorkjong/news-digest/main/src/{fn}'
    !wget $url

%pip install discord.py

In [17]:
# @title Initialize
import clip
import hashtag

In [18]:
# @title Setting Environment Variables for Security Data
from google.colab import userdata

os.environ['BOT_TOKEN'] = userdata.get('NEWS_DIGEST.BOT_TOKEN')


In [19]:
# @title Discord


import discord
import asyncio

import nest_asyncio
nest_asyncio.apply()


TOKEN = os.environ['BOT_TOKEN']


def post_via_bot(channel_id: int, message: str):
    """
    Sends a message to a specified Discord announcement channel and publishes it.
    Automatically logs out after sending.
    Parameters:
        channel_id (int): The target channel ID
        message (str): The content to post
    """

    class PublishBot(discord.Client):
        def __init__(self, channel_id, message, **kwargs):
            super().__init__(**kwargs)
            self.channel_id = channel_id
            self.message = message

        async def on_ready(self):
            #print(f"✅ Bot {self.user} is online. Sending message to channel {self.channel_id}...")

            #for guild in self.guilds:
            #    print(f"Guild: {guild.name} (ID: {guild.id})")
            #    for channel in guild.text_channels:
            #        print(f"  - Channel: {channel.name} (ID: {channel.id})")

            channel = self.get_channel(self.channel_id)
            #print(f"📡 get_channel returned: {channel}")

            if channel:
                try:
                    sent = await channel.send(self.message)
                    await sent.publish()
                    print(f"✅ Message published successfully to channel {self.channel_id}.")
                except discord.Forbidden:
                    print(f"⚠️ Missing permissions to publish in channel {self.channel_id}.")
                except Exception as e:
                    print(f"❌ Error while sending to channel {self.channel_id}: {e}")
            else:
                print(f"❌ Channel {self.channel_id} not found.")

            #print("🛑 Task completed. Bot is logging out.")
            await self.close()

    intents = discord.Intents.default()
    intents.guilds = True
    intents.messages = True
    bot = PublishBot(channel_id, message, intents=intents)

    asyncio.run(bot.start(TOKEN))

### Execute Actions Step by Step

In [20]:
#@title Step 1. Pick a **period** of the news { run: "auto", display-mode: "form" }
period = "Today" #@param ["Today", "Yesterday", "Recent 2 Days", "Recent 7 Days"]

def news_today():
    from datetime import datetime
    today = datetime.today().strftime('%Y_%m_%d')
    last = clip.get_recent_journal_filenames(1)[0][:-3]
    if today != last:
        print(f'last date: {last}')
        print(f'today: {today}')
    return clip.get_latest_journal()

def news_yesterday():
    fns = clip.get_recent_journal_filenames(2)
    return clip.get_journal(fns[-2])

def news_recent2days():
    return clip.merge_recent_journals(days=2)

def news_recent7days():
    return clip.merge_recent_journals(days=7)

period2func = {
    'Today': news_today,
    'Yesterday': news_yesterday,
    'Recent 2 Days': news_recent2days,
    'Recent 7 Days': news_recent7days,
}

content = period2func[period]()

import ipywidgets as widgets
output = widgets.Output()
with output:
    print(content)

In [21]:
#@title Step 2. Discord Notify { display-mode: "form" }
mock_mode = True #@param {type:"boolean"}
show_headings = False #@param {type:"boolean"}

import ipywidgets as widgets
import time


subscriptions_daily = (
    (('Tesla & SpaceX; Vehicle',), 1389870333934436374),
    (('Tech Industry',), 1389878724270493756),
    (('Crypto',), 1389879470223003780),
    (('Finance',), 1389879306431500328),
    (('IT',), 1389878977006669957),
    (('Taiwan',), 1389958978158465146),
)

subscriptions_weekly = (
    (('Science & Technology',), 1390201027772289094),
    (('Health & Food',), 1390202280074023062),
)

period2subscriptions = {
    'Today': subscriptions_daily,
    'Yesterday': subscriptions_daily,
    'Recent 2 Days': subscriptions_daily,
    'Recent 7 Days': subscriptions_weekly,
}

def create_outputs(subscriptions):
    topics = [str(t) for t, _ in subscriptions]
    tab = widgets.Tab()
    outputs = [widgets.Output() for name in topics]
    tab.children = outputs
    for i, t in enumerate(topics):
        tab.set_title(i, t)
    display(tab)
    return outputs


def notify():
    subscriptions = period2subscriptions[period]
    if mock_mode:
        outputs = create_outputs(subscriptions)
    for i, (topics, ch_id) in enumerate(subscriptions):
        headings = [topic for topic in topics if not topic.startswith('#')]
        tags = [topic for topic in topics if topic.startswith('#')]
        categories = headings
        if not categories and tags:
            categories = clip.get_categories(content)
        if tags:
            lines = clip.get_lines_of_categories(categories, content, True, True)
            lines = hashtag.get_lines_with_any_hashtags(lines, tags)
            with_headings = True if headings and show_headings else False
            lines = clip.get_lines_of_categories(categories, '\n'.join(lines), False, with_headings)
        else:
            lines = clip.get_lines_of_categories(categories, content, False, show_headings)
        if not lines:
            continue
        text = clip.markdown_to_readable('\n'.join(lines))
        message = f'\n{text}'

        if mock_mode:
            with outputs[i]:
                print(message)

        if not mock_mode:
            post_via_bot(ch_id, message)
        if not mock_mode:
            time.sleep(1)

notify()

### Misc

In [22]:
import requests


def notify_message(message: str, webhook_url: str):
    data = {
        "content": message
    }
    response = requests.post(webhook_url, json=data)
    if response.status_code == 204:
        print("✅ Message successfully sent to Discord.")
    else:
        print(f"❌ Failed to send message. Status code: {response.status_code}, Response: {response.text}")

